## Behavioral Cloning Project

The goals / steps of this project are the following:

+ Use the simulator to collect data of good driving behavior
+ Build, a convolution neural network in Keras that predicts steering angles from images
+ Train and validate the model with a training and validation set
+ Test that the model successfully drives around track one without leaving the road
+ Summarize the results with a written report

## Model Architecture and Training Strategy
### 1) An appropriate model architecture has been employed

My model embraces transfer learning and as a result is motivated by [a paper from Nvidia](https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf). The model includes RELU layers to introduce nonlinearity, and the data is normalized in the model using a Keras lambda layer.

**Layers:**

+ **Lambda** - normalization layer
+ **Convolution2D** - convolution with 5x5 & 3x3 kernels, padding valid and RELU activation.
+ **MaxPooling2D** - useful to reduce dimensions
+ **Cropping2D** - remove irrelevant parts of image
+ **Dropout** - prevents overfiting
+ **Flatten** - converting output of convolutional part of the CNN into a 1 dimensional feature vector
+ **Dense** - regression output (steering angle)

In [ ]:
ch, row, col = 3, 160, 320

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(row,col,ch)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(64,3,3, activation="relu"))
model.add(Convolution2D(64,3,3, activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=args.lrate)) 

### 2) Attempts to reduce overfitting in the model
The model contains dropout layers in order to reduce overfitting.

The model was trained and validated on different data sets to ensure that the model was not overfitting. The model was tested by running it through the simulator and ensuring that the vehicle could stay on the track.

### 3) Model parameter tuning
The model used an adam optimizer and I experimented with various values for learning rate and settled on .0001 since it resulted in the least validation loss.

### 4) Appropriate training data
Training data was chosen to keep the vehicle driving on the road. I used a combination of center lane driving, recovering from the left and right sides of the road, driving counter clockwise and recovery driving to help the vehicle learn how to recover from a bad steering angle.

For details about how I created the training data, see the next section.

## Model Architecture and Training Strategy

### 1) Solution Design Approach

The overall strategy for deriving a model architecture was iterative. My first step after downloading the simulator, was to drive the vehicle manually for a few laps to get a feel for the controls and get better at track 1. My ability to manually drive the vehicle would be porportional to the quality of the training samples produced. I recorded one lap for an initial end to end test scenario prior to implementing the full CNN architecture.

The second step was to use a very simple convolution neural network sequential model with one flat and dense layers. I thought this model might be appropriate since I was primarily interested in and end-to-end test scenario to define a workflow for training and get familiar with the tools provided.

After training this initial model, the vehicle performed as expected, poorly. The next step was to gather more training data and perform image processing. In addition, I downloaded the training dataset available from Udacity for augmentation as well. Here are some processing steps taken:

+ cropping images to remove top part of the image which isn't necessary
+ using multiple cameras in addition to center image
+ flip images horizontally

In order to gauge how well the model was working, I split my image and steering angle data into a training and validation set. I found that my first model had a low mean squared error on the training set but a high mean squared error on the validation set. This implied that the model was overfitting.

To combat the overfitting, I modified the model to include dropout.

Then I looked at the Nvidia paper (mentioned above) for a reference architecture and integrated it into this project.

I ran into a memory error after a sufficiently large number of training samples, so I refactored my model training to use generators and train on batches of images instead of having to load the entire data into memory.

The final step was to run the simulator to see how well the car was driving around track one. There were a few spots where the vehicle fell off the track. Inorder to improve the driving behavior in these cases, I recorded some recovery driving images and retrained the model. 

At the end of the process, the vehicle is able to drive autonomously around the track without leaving the road.

### 2) Final Model Architecture

![CNN Architecture](./assets/cnn_architecture.png)

### 3) Creation of the Training Set & Training Process
To capture good driving behavior, I first recorded two laps on track one using center lane driving. Here is an example image of center lane driving:

![CNN Architecture](./assets/center_2017_03_15_07_36_35_267.jpg)

I then recorded the vehicle recovering from the left side and right sides of the road back to center so that the vehicle would learn to stay in the lane and be more generalizable. These images show what a recovery looks like starting from ... :

![CNN Architecture](./assets/left_2017_03_19_14_10_04_330.jpg)

![CNN Architecture](./assets/right_2017_03_19_14_09_47_312.jpg)

After the collection process, I had 12K number of data points.  I finally randomly shuffled the data set and put Y% of the data into a validation set.

I used this training data for training the model. The validation set helped determine if the model was over or under fitting. Here are the model hyperparameter values:

+ learning rate: .0001
+ number of epochs: 6
+ batch size: 32